In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df=pd.read_csv("8358_1.csv")

In [ ]:
df.head()

# Preprocessing and Feature engineerin

In [ ]:
df.info()

**this dataset is full of nans lets see how to handel them**

In [ ]:
df.isnull().sum()

**I will drop:**

**id and menuepageurl and keys and name since they are useless**

**city and country and province since I have longitute and latitude**

**also menues date seen sice the data won't effect the clustering(even if prices have chainged It won't matter)**

**and menue description since it has a lot of unreplaceable datas**

**and postal code(it would be usefull but we have longitude and latitude also it contains characters beside numbers)**

In [ ]:
df=df.drop(["id","address","city","country","keys","menuPageURL","menus.dateSeen","menus.description","province","postalCode"],axis=1)

In [ ]:
df.info()

In [ ]:
len(list(df["categories"].unique()))

**I will drop those columns which have a lot of unique labels**

In [ ]:
df=df.drop("categories",axis=1)

In [ ]:
df["menus.currency"].value_counts()

**since all of them are the same I will drop it**

In [ ]:
df=df.drop("menus.currency",axis=1)

In [ ]:
len(list(df["menus.name"].unique()))

In [ ]:
df=df.drop("menus.name",axis=1)

In [ ]:
len(list(df["name"].unique()))

In [ ]:
df=df.drop("name",axis=1)

In [ ]:
df["priceRangeCurrency"].value_counts()

In [ ]:
df=df.drop("priceRangeCurrency",axis=1)

In [ ]:
df.info()

**In order to find and replace nans of price range min and max I should search the realtion in a correlation**

In [ ]:
sns.heatmap(df.corr(),annot=True)

In [ ]:
sns.heatmap(df.isnull(),cmap="plasma")

**there is lots of missing values I have to drop them**
**I dont have enough data for knn to predict them most of them are missing values**

In [ ]:
df=df.dropna()

In [ ]:
df.info()

In [ ]:
df.describe()

# PCA

(I did this part to do just some experiment the main clustering at the end)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca=PCA(2)

In [ ]:
df_pca=pd.DataFrame(pca.fit_transform(df))

In [ ]:
sns.scatterplot(df_pca[0],df_pca[1])

**those points are missing values let's dropt them**

In [ ]:
df_pca = df_pca.loc[(df_pca[0] < 300) & (df_pca[1] < 50)]

In [ ]:
sns.scatterplot(df_pca[0],df_pca[1])

**So dataset has 4 or even 5 clusters**

**Also kmeans won't act well on this dataset because of the logic, DBSCAN would be the best**

# Kmeans

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
wcss=[]
for i in range(2,20):
    kmeans=KMeans(n_clusters=i)
    kmeans.fit(df_pca)
    wcss.append(kmeans.inertia_)

**------elbow method-----**

In [ ]:
plt.plot(range(2,20),wcss,"bx-")
plt.xticks(range(2,20))

In [ ]:
kmeans=KMeans(n_clusters=5)

In [ ]:
label_kmeans=kmeans.fit(df_pca).labels_

**so the elbow is at 5**

In [ ]:
centroid=pd.DataFrame(kmeans.cluster_centers_)

In [ ]:
sns.scatterplot(df_pca[0],df_pca[1],hue=label_kmeans)
sns.scatterplot(centroid[0],centroid[1])

# DBSCAN
(we could also use the main data without removing missing values because this model handels missing values)

In [ ]:
from sklearn.cluster import DBSCAN

**Since the distance between data clusters is too high I set the eps to a higher rate**

In [ ]:
dbs=DBSCAN(eps=5)

In [ ]:
dbs.fit(df_pca)

In [ ]:
label_dbs=dbs.labels_

In [ ]:
sns.scatterplot(df_pca[0],df_pca[1],hue=label_dbs,palette="viridis")

# SpectralClustering

In [ ]:
from sklearn.cluster import SpectralClustering

In [ ]:
sc=SpectralClustering(n_clusters=5)

In [ ]:
label_sc=sc.fit_predict(df_pca)

In [ ]:
sns.scatterplot(df_pca[0],df_pca[1],hue=label_sc,palette="viridis")

**This is the best result**

# Optic

In [ ]:
from sklearn.cluster import OPTICS

In [ ]:
optics=OPTICS(eps=5)

In [ ]:
label_opt=optics.fit_predict(df_pca)

In [ ]:
sns.scatterplot(df_pca[0],df_pca[1],hue=label_opt,palette="viridis")

# Gaussian mixture

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
gm=GaussianMixture(n_components=5)

In [ ]:
label_gm=gm.fit_predict(df_pca)

In [ ]:
sns.scatterplot(df_pca[0],df_pca[1],hue=label_gm,palette="viridis")

In [ ]:
gm=GaussianMixture(n_components=4)
label_gm=gm.fit_predict(df_pca)
sns.scatterplot(df_pca[0],df_pca[1],hue=label_gm,palette="viridis")

# AgglomerativeClustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
agc=AgglomerativeClustering(n_clusters=5)

In [ ]:
label_agc=agc.fit_predict(df_pca)

In [ ]:
gm=GaussianMixture(n_components=5)
label_gm=gm.fit_predict(df_pca)
sns.scatterplot(df_pca[0],df_pca[1],hue=label_agc,palette="viridis")

# Clusterin

In [ ]:
kmeans=KMeans(n_clusters=4)

In [ ]:
df["label"]=kmeans.fit_predict(df)